<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import plotly as pl
import plotly.express as px
from collections import Counter

In [1]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = ''

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

### 1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.vacancies
'''

In [5]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,49197


### 2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.employers 
'''

In [7]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,23501


### 3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.areas 
'''

In [9]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,1362


### 4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.industries
'''

In [11]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,294


***

# выводы по предварительному анализу данных
*1. Общее количество вакансий 49197.\
2. Общее количество работодателей 23501.\
3. Количество уникальных регионов 1362.\
4. Количество сфер деятельности 294.*

*В среднем от каждого работодателя есть 2 вакансии.\
В среднем на каждый регион приходится по 36 вакансий и 18 работодателей*

------

*Дополнительно посмотрим на количество вакансий в которых заполены все столбцы. Таких примерно 12 тысяч, что составляет около 24% от числа всех вакансий.*

In [12]:
# Запрашиваем объединенную таблицу, удаляем отсутствующие значения и выводим количество получившихся вакансий

query = f'''
SELECT distinct *
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
JOIN public.employers e ON v.employer_id = e.id
JOIN public.employers_industries ei ON v.employer_id = ei.employer_id
JOIN public.industries i ON ei.industry_id = i.id
'''
pd.read_sql_query(query, connection).dropna().shape[0]

D:\Temp\ipykernel_15508\3248067823.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection).dropna().shape[0]


11994

# Юнит 4. Детальный анализ вакансий

### 1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query = f'''
SELECT a.name, count(*)
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
GROUP BY a.name
ORDER BY count(*) desc
LIMIT 5
'''

In [14]:
# результат запроса
pd.read_sql_query(query,connection)

D:\Temp\ipykernel_15508\1941775883.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query,connection)


,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


### 2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query = f'''
SELECT count(*) "Количество"
FROM public.vacancies
WHERE (salary_from is not null) OR (salary_to is not null)
'''

In [16]:
# результат запроса
pd.read_sql_query(query,connection)

D:\Temp\ipykernel_15508\1941775883.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query,connection)


,Количество
0,24073


### 3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query = f'''
SELECT 'Верхняя ветка' "Граница", round(avg(salary_to)) "Средняя ЗП"
FROM public.vacancies

UNION ALL

SELECT 'Нижняя ветка', round(avg(salary_from))
FROM public.vacancies
'''

In [18]:
# результат запроса
pd.read_sql_query(query,connection)

D:\Temp\ipykernel_15508\1941775883.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query,connection)


,Граница,Средняя ЗП
0,Верхняя ветка,110537.0
1,Нижняя ветка,71065.0


### 4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query = f'''
SELECT  schedule||'--'||employment combination, 
count(*)
FROM public.vacancies
GROUP BY schedule, employment
ORDER BY 2 desc
'''

In [20]:
# результат запроса
pd.read_sql_query(query,connection)

D:\Temp\ipykernel_15508\1941775883.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query,connection)


,combination,count
0,Полный день--Полная занятость,35367
1,Удаленная работа--Полная занятость,7802
2,Гибкий график--Полная занятость,1593
3,Удаленная работа--Частичная занятость,1312
4,Сменный график--Полная занятость,940
5,Полный день--Стажировка,569
6,Вахтовый метод--Полная занятость,367
7,Полный день--Частичная занятость,347
8,Гибкий график--Частичная занятость,312
9,Полный день--Проектная работа,141


### 5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query = f'''
SELECT experience,
count(*)
FROM public.vacancies
GROUP BY experience
ORDER BY 2
'''

In [22]:
# результат запроса
pd.read_sql_query(query,connection)

D:\Temp\ipykernel_15508\1941775883.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query,connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

# выводы по детальному анализу вакансий

*1. На передних местах по количеству вакансий находятся города-миллионники. И это не удивительно, в них сосредоченно огромное количество компаний, как больших так и маленьких. *\
*2. В <u>51%</u> всех вакансий не заполнена частично или полностью информация о предпологаемой заработной плате. 
У работадателя существует множество причин не указывать заработную плату и вот несколько причин почему:*\
*- Зарплата может зависеть от уровня кандидата;*\
*- Объем работ заранее не определен, поэтому расчет будет при личном собеседивании;*\
*- Зарплата может не соответствовать текущему уровню зп для этой профессии на рынке, но есть другие бонусы;*\
*- и т.д.*\
*3. Средняя заработная плата по всем вакансиям в районе 90 тысяч.*\
*4. По данным о количестве вакансий для комбинации график-трудоустройство на первом месте находится Полный день-полная занятость. *\
*5. Работодатель в большинстве случаев (53%) ищет сотрудников с минимальных (1-3 года) опытом работы.*

Большое количество пропущенных значений столбцов **зарплатные вилки** нужно обработать. Лучше добавить столбец со средней заработной платой, а зарплатные вилки удалить. Отсутствующие значения заполнить средним значением по данной группе.

------------

Дополнительно можно посмотреть на самые восстребованные вакансии, а так же, \
Какие навыки чаще всего указываются работодателем.

In [23]:
query = f'''
select distinct v.name, count(*)
from public.vacancies v
group by v.name
order by 2 desc
limit 10
'''

pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\1668844238.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,name,count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


In [24]:
query = f'''
SELECT v.key_skills
FROM public.vacancies v
WHERE v.key_skills IS NOT NULL
'''
skill_dct = {}                                                        # Словарь для подсчета количеста навыков

def key_skillz_counter(x: str) -> str:
    # С помощью функции Counter считаем количество навыков и добавляем их в словарь
    for key, value in Counter(x.split('\t')).items():
        skill_dct[key] = skill_dct.get(key, 0) + value
    return x
    
pd.read_sql_query(query, connection)['key_skills'].apply(key_skillz_counter)

list_ = sorted(skill_dct.items(), key=lambda x: x[1], reverse=True)   # Сортируем, разделяем на 2 переменные и формируем Series
skills = [i[0] for i in list_]
count = [i[1] for i in list_]

pd.Series(count, index=skills).head(10)

D:\Temp\ipykernel_15508\181257613.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)['key_skills'].apply(key_skillz_counter)


SQL                     6977
Git                     4731
Английский язык         4525
Работа в команде        3923
Linux                   3747
JavaScript              3625
Python                  3445
Управление проектами    2764
1С программирование     2647
PostgreSQL              2452
dtype: int64

# Юнит 5. Анализ работодателей

### 1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [25]:
# текст запроса
query = f'''
(                                             /*Запрос первого места*/
SELECT e.name,
count(v.id)
FROM public.employers e
JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY e.id
ORDER BY 2 desc
LIMIT 1
)
UNION ALL
(                                             /*Запрос пятого места*/
SELECT e.name,
count(v.id)
FROM public.employers e
JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY e.id
ORDER BY 2 desc
OFFSET 4 LIMIT 1
)
'''
            

In [26]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,name,count
0,Яндекс,1933
1,Газпром нефть,331


### 2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [27]:
# текст запроса
query = f'''
SELECT ae.name "Регион",
"Количество работодателей",
"Количество вакансий" 
FROM
(                                             
SELECT a.name, 
count(e.id) "Количество работодателей"
FROM public.areas a
LEFT JOIN public.employers e ON a.id = e.area
GROUP BY a.id 
) ae
JOIN 
(
SELECT a.name, 
count(v.id) "Количество вакансий" 
FROM public.areas a 
LEFT JOIN public.vacancies v 
ON a.id = v.area_id
GROUP BY a.id 
) av
ON ae.name = av.name
ORDER BY 3, 2 DESC
'''

In [28]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


### 3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [29]:
# текст запроса
query = f'''
SELECT e.name,
count(DISTINCT v.area_id)
FROM public.vacancies v
JOIN public.employers e ON e.id = v.employer_id
GROUP BY e.id
ORDER BY 2 DESC
'''

In [30]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


### 4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [31]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
WHERE industry_id IS NULL
'''

In [32]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,8419


### 5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [33]:
# текст запроса
query = f'''
SELECT e.name 
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
GROUP BY e.id
HAVING count(*) = 4
ORDER BY 1
OFFSET 2 LIMIT 1
'''

In [34]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,name
0,2ГИС


### 6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.

In [35]:
# текст запроса
query = f'''
SELECT count(DISTINCT e.id)
FROM public.employers e
JOIN public.employers_industries ei ON e.id = ei.employer_id
JOIN public.industries i ON ei.industry_id = i.id
LEFT JOIN public.vacancies v ON v.employer_id = e.id
WHERE i.name like '%Разработка программного обеспечения%'
'''

# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\2168848807.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,3553


In [36]:
# Дополнительно взглянем на имена компаний с наибольшим количеством вакансий в сфере - Разработка програмного обеспечения 
query = f'''                                      
SELECT e.name, count(DISTINCT v.id)
FROM public.employers e
JOIN public.employers_industries ei ON e.id = ei.employer_id
JOIN public.industries i ON ei.industry_id = i.id
LEFT JOIN public.vacancies v ON v.employer_id = e.id
WHERE i.name like '%Разработка программного обеспечения%'
GROUP BY e.id
ORDER BY 2 DESC
LIMIT 10
'''

pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\2163303907.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,name,count
0,Тинькофф,444
1,DataArt,247
2,Совкомбанк Технологии,204
3,IBS,175
4,Тензор,155
5,Andersen,155
6,VK,150
7,ЛАНИТ,117
8,Лига Цифровой Экономики,110
9,Softline,108


### 7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [37]:
# код для получения списка городов-милионников
url = r'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
rows = soup.find('table', class_='standard sortable').find('tbody').find_all('tr')

headers = pd.Series(['№', 'Город', 'Население, тыс. чел. (на 1 января 2024)', 'Население, тыс. чел. (перепись-2021)',
                     'Прирост относительно переписи-2021', 'Население, тыс. чел. (перепись-2020)'])
lst = []
for row in rows:
    values = row.find_all('td')
    if values:
        try:
            lst.append([v.text if i != 4 else v['data-sort-value'] + ' %' for i, v in enumerate(values)])
        except Exception:
            continue

df = pd.DataFrame(lst, columns=headers)
df.set_index('№',inplace=True)
df

,Город,"Население, тыс. чел. (на 1 января 2024)","Население, тыс. чел. (перепись-2021)",Прирост относительно переписи-2021,"Население, тыс. чел. (перепись-2020)"
№,,,,,
1,Москва,13 150,13 010,13.1 %,11 504
2,Санкт-Петербург,5 598,5 602,14.8 %,4 880
3,Новосибирск,1 634,1 634,10.9 %,1 474
4,Екатеринбург,1 536,1 544,14.4 %,1 350
5,Казань,1 319,1 309,14.4 %,1 144
6,Красноярск,1 205,1 188,21.6 %,977
7,Нижний Новгород,1 205,1 226,0.2 %,1 251
8,Челябинск,1 177,1 190,5.3 %,1 130
9,Уфа,1 163,1 145,7.8 %,1 062


In [38]:
# текст запроса
query = f'''
WITH tbl AS (
SELECT a.name,
count(*) 
FROM public.employers e
JOIN public.vacancies v ON e.id = v.employer_id
JOIN public.areas a ON a.id = v.area_id 
WHERE e.name like 'Яндекс' AND a.name in {tuple(city for city in df['Город'])}
GROUP BY a.name)

SELECT * FROM tbl

UNION ALL

SELECT 'Total',
sum(tbl.count)
FROM tbl
ORDER BY 2
'''

In [39]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,name,count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

# выводы по анализу работодателей
1. По количеству вакансий на первом месте находится компания 'Яндекс'.
2. Города-миллионники, которые лидировали по количеству вакансий, так же лидируют по количеству работодателей. Отдельно можно отметить регионы в которых большое количество работодателей, но отсутствуют вакансии, это можно отнести к тому, что у работодателя просто нет открытых вакансий или в вакансии не указан работодатель.
3. По количеству регионов, в которых компания публикует свои записи, так же лидирует 'Яндекс'.
4. Из 23501 вакансий в 8419 не указана сфера деятельности (примерно 36%).
5. Компания у которой больше всего вакансий и имеет 4 сферы деятельности - 2ГИС.
6. Примерно у 15% всех работодателей в качестве сферы деятельности указана Разработка программного обеспечения. На удивление, у компании 'Яндекс', по данным, находящейся на первом месте по количеству вакансий, в сфере деятельности <u>НЕ</u> указана Разработка программного обеспечения.
7. Общее количество вакансий от компании 'Яндекс' , для городов-миллионников, составляет 485. Из них наименьшее количество в Омске, наибольшее в Москве.

В таблице с регионами, присутствую как поселки городского типа, так и города, и даже страны. Например там есть Москва и Россия как разные регионы.\
Данные значения, лучше, в дальнейшем удалить или представить в виде городов, чтобы все значения соответствовали одной группе.

# Юнит 6. Предметный анализ

### 1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [40]:
# текст запроса
query = f'''
SELECT count(*)
FROM 
(
select lower(v.name) "name"          /* Посчитаем количество из таблицы с именами в нижнем регистре*/
FROM public.vacancies v
) t
WHERE t.name LIKE '%data%' OR t.name LIKE '%данн%'
'''

In [41]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15508\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,1771


### 2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [37]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.vacancies v
WHERE 
(LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(UPPER(v.name) LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%')
AND
(lower(v.name) LIKE '%junior%' OR
v.experience LIKE '%Нет опыта%' OR
v.employment LIKE '%Стажировка%')
'''

In [38]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_11388\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,51


### 3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [42]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.vacancies v
WHERE 
(LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(UPPER(v.name) LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%')
AND
(UPPER(v.key_skills) LIKE '%SQL%' OR
LOWER(v.key_skills) LIKE '%postgres%')
'''

In [43]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_13528\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,229


### 4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [8]:
# текст запроса
query = f'''
SELECT count(*)
FROM public.vacancies v
WHERE 
(LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(UPPER(v.name) LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%')
AND
LOWER(v.key_skills) LIKE '%python%' 
'''

In [9]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_15776\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,count
0,357


### 5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
# текст запроса
query = f'''
SELECT v.key_skills "skills"
FROM public.vacancies v
WHERE 
LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(v.name LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR 
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%'
'''

In [47]:
# результат запроса
pd.read_sql_query(query, connection)['skills'].dropna().apply(lambda x: len(x.split('\t'))).agg('mean').round(2)

D:\Temp\ipykernel_13528\1714276914.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)['skills'].dropna().apply(lambda x: len(x.split('\t'))).agg('mean').round(2)


np.float64(6.41)

### 6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

**Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.**

In [4]:
# текст запроса
query = f'''
SELECT DISTINCT criteria.experience,
ROUND(                                                                       /* Задаем условия при которых формируется средняя заработная плата*/
    AVG( 
        CASE
            WHEN criteria.salary_from IS NULL THEN criteria.salary_to
            WHEN criteria.salary_to IS NULL THEN criteria.salary_from
            ELSE (criteria.salary_from + criteria.salary_to) / 2
        END
        )
    ) "Средняя ЗП"
FROM
(
SELECT *
FROM public.vacancies v
WHERE 
LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(v.name LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%'
) criteria
WHERE criteria.salary_from IS NOT NULL OR criteria.salary_to IS NOT NULL     /* Выбираем только те строки в которых заполнен хотябы 1 столбец с зарплатой*/
GROUP BY criteria.experience
HAVING criteria.experience LIKE '%От 3 до 6 лет%'
'''

In [5]:
# результат запроса
pd.read_sql_query(query, connection)

D:\Temp\ipykernel_19412\763091750.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,experience,Средняя ЗП
0,От 3 до 6 лет,243115.0


***

# выводы по предметному анализу

1. По первому пунку, подходящих вакансий, в которых встречается части слова 'data' и 'данн', 1771.
2. Во втором пункте, мы добавляем дополнительные параметры фильтрации, что сокращает наш диапазон подходящих вакансий до 536. Здесь же выбираем из них вакансии для уровня <u>junior</u> - получается 51 вакантная должность.
3. Количество вакансий в которых в качестве ключевого навыка указаны 'sql' или 'postgresql' 229. 
4. Количество вакансий в которых в качестве ключевого навыка указаны 'python' - 357.
5. В среднем количество ключевых навыков которые указывает работодатель примерно равно 6.
6. Средняя заработная плата для профессии Data Science, со стажем от 3 до 6 лет составляет 243115.


# Общий вывод по проекту

## подведем итог исследования, обобщите выводы

В общем можно отметить большое количество пропущенной информации в БД. Лишь 24% всех вакансий полностью заполнены.\
Самой восстребованной вакансией является Системный администратор, а самым восстребованным навыком - 'python'.\
Количество указанных ключевых навыков не влияют на заработную плату.\
Работодателей достаточно много и если посмотреть в общем на количество их вакансий в больших городах, то их отношение почти равняется. Напротив в городах поменьше количество вакансий намного превосходит количество работодателей.

## здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

**Нашей основной целью, по условию задачи, является создание модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist.\
Что бы понять, какие вакансии являются более подходящими, нам нужно в первую очередь определить <u>наиболее выжные признаки для соискателя</u>. Для примера,  возьмем исследование сервиса по поиску высокооплачиваемой работы SuperJob. В опросе приняли участие представители работодателей и экономически активные россияне из всех округов страны.**

**Место проведения опроса: Россия, все округа\
Населенных пунктов: 245\
Время проведения: 21—29 июля 2022 года\
Исследуемая совокупность: менеджеры по персоналу и другие представители кадровых служб предприятий и организаций, ответственные за подбор персонала\
Размер выборки: 500 респондентов\
Вопрос: ***«Какие три фактора, на ваш взгляд, являются самыми важными для соискателей при выборе места работы?»*** (без учета респондентов, затруднившихся с ответом)**

In [50]:
# В таблице присутствуют данные по опросам за предыдущие годы
url = r'https://www.superjob.ru/research/articles/113593/top-3-prioritetnyh-faktorov-pri-vybore-raboty-soiskatelyami/#:~:text=%D0%9D%D0%B0%D0%B8%D0%B1%D0%BE%D0%BB%D0%B5%D0%B5%20%D0%B2%D0%B0%D0%B6%D0%BD%D1%8B%D0%BC%20%D1%84%D0%B0%D0%BA%D1%82%D0%BE%D1%80%D0%BE%D0%BC%20%D0%BF%D1%80%D0%B8%20%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%B5,28%25%20%E2%80%94%20%D0%BA%D0%BE%D0%BC%D1%84%D0%BE%D1%80%D1%82%D0%BD%D0%B0%D1%8F%20%D0%BF%D1%81%D0%B8%D1%85%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F%20%D1%81%D1%80%D0%B5%D0%B4%D0%B0.'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
rows = soup.find('table', attrs={'class': 'table', 'table-id': '62e3b46dadbb7'})

headers = [t.text.strip() for t in rows.find('thead').find('tr').find_all('td')]
body = [[i.text.strip() for i in row.find_all('td')] for row in rows.find('tbody').find_all('tr')]

df = pd.DataFrame(body, columns=headers)
df.index = df.index + 1
df[:3]

,Вариант ответа,2022,2021,2019,2016,2015
1,Высокий уровень оплаты труда,71%,61%,74%,74%,69%
2,Стабильность/устойчивость компании,52%,46%,50%,54%,55%
3,Удобное расположение,39%,35%,38%,33%,28%


__На первом месте__ — **Высокий уровень оплаты труда (71%).**\
__На втором месте__ — **Стабильность/устойчивость компании (52%),**\
__На третьем__ — **Удобное расположение (39%).**\
\
**По данным показателям выберем для своей модели 3 аналогичных признака:**\
**1. В наших данных, в признаке заработная плата, очень много пропущенных значений(452 из 536 подходящих для нас вакансий!) это примерно <u>84%</u>, что является большой проблемой т.к данный признак является ключевым.**

In [97]:
# Количество вакансий в которых пропущенны значения зарплатной вилки для вакансий Data Science, разделенные по группам

query = f'''
SELECT 
with_zp.experience, 
with_zp."Кол-во вакансий с ЗП",
without_zp."Кол-во вакансий без ЗП"
FROM
(
SELECT distinct v.experience,
COUNT(*) "Кол-во вакансий без ЗП"
FROM public.vacancies v
WHERE 
(LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(UPPER(v.name) LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%')
AND (v.salary_from IS NULL AND v.salary_to IS NULL)
GROUP BY v.experience
ORDER BY 2
) without_zp

JOIN 

(
SELECT distinct v.experience,
COUNT(*) "Кол-во вакансий с ЗП"
FROM public.vacancies v
WHERE 
(LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(UPPER(v.name) LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%')
AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
GROUP BY v.experience
ORDER BY 2
) with_zp ON without_zp.experience = with_zp.experience
'''

pd.read_sql_query(query, connection)

D:\Temp\ipykernel_13528\3321744322.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,experience,Кол-во вакансий с ЗП,Кол-во вакансий без ЗП
0,Нет опыта,7,21
1,Более 6 лет,4,39
2,От 3 до 6 лет,41,187
3,От 1 года до 3 лет,32,205



**Попробуем создать дополнительный столбец со средней заработной платой, которая будет высчитываться аналогично задаче 6 в Юните 6
, только пропущенные значения мы будем заполнять средним сначением для этой группы.**

In [44]:
query = f'''
SELECT DISTINCT criteria.experience,
ROUND(                                                                       /* Задаем условия при которых формируется средняя заработная плата*/
    AVG( 
        CASE
            WHEN criteria.salary_from IS NULL THEN criteria.salary_to
            WHEN criteria.salary_to IS NULL THEN criteria.salary_from
            ELSE (criteria.salary_from + criteria.salary_to) / 2
        END
        )
    ) "Среднее"
FROM
(
SELECT *
FROM public.vacancies v
WHERE 
LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(v.name LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%'
) criteria
WHERE criteria.salary_from IS NOT NULL OR criteria.salary_to IS NOT NULL     /* Выбираем только те строки в которых заполнен хотябы 1 столбец с зарплатой*/
GROUP BY criteria.experience
ORDER BY 2
'''

''' 
Данные по среднему значению заработной платы разделенные на группы. В данных не оказалось значений для группы 'Более 6 лет', это означает, что
все вакансии с опытом работы 'Более 6 лет' не имеют ни нижней, ни верхней граница оплаты труда. 
'''
df = pd.read_sql_query(query, connection) 

''' 
Чтобы заполнить недостающие значения для группы 'Более 6 лет' вычислим тенденцию роста ЗП.
Из каждой группы, кроме первой, вычисляем процент увеличения ЗП по отношению к первой группе.
Из них получаем процент увелчиения ЗП для третьей группы и испольуем его для формирования среднего значения по группе 'Более 6 лет'
'''
perc_grow1 = round(df['Среднее'][1] / df['Среднее'][0] * 100 - 100)         
perc_grow2 = round(df['Среднее'][2] / df['Среднее'][0] * 100 - 100)
perc_grow3 = (perc_grow2 - perc_grow1) + perc_grow2


query2 = f'''
SELECT *, 
        CASE
            WHEN criteria.salary_from IS NULL AND criteria.salary_to IS NULL THEN {df['Среднее'][0] * (perc_grow3 / 100 + 1)}
            
            WHEN criteria.salary_from IS NULL AND criteria.experience LIKE 'Нет опыта' THEN {df['Среднее'][0]}
            WHEN criteria.salary_from IS NULL AND criteria.experience LIKE 'От 1 года до 3 лет' THEN {df['Среднее'][1]}
            WHEN criteria.salary_from IS NULL AND criteria.experience LIKE 'От 3 до 6 лет' THEN {df['Среднее'][2]}
            
            WHEN criteria.salary_to IS NULL AND criteria.experience LIKE 'Нет опыта' THEN {df['Среднее'][0]} 
            WHEN criteria.salary_to IS NULL AND criteria.experience LIKE 'От 1 года до 3 лет' THEN {df['Среднее'][1]} 
            WHEN criteria.salary_to IS NULL AND criteria.experience LIKE 'От 3 до 6 лет' THEN {df['Среднее'][2]} 
            ELSE (criteria.salary_from + criteria.salary_to) / 2
        END "Средняя ЗП"
FROM
(
SELECT *
FROM public.vacancies v
WHERE 
LOWER(v.name) LIKE '%data scientist%' OR
LOWER(v.name) LIKE '%data science%' OR
LOWER(v.name) LIKE '%исследователь данных%' OR
(v.name LIKE '%ML%' AND NOT UPPER(v.name) LIKE '%HTML%') OR
LOWER(v.name) LIKE '%machine learning%' OR
LOWER(v.name) LIKE '%машинн%обучен%'
) criteria
'''

# DataFrame с вакансиями Data Science и дополнительным столбом со средней заработной платой
df_bd = pd.read_sql_query(query2, connection)

# Удаляем менее информативные столбцы salary_from и salary_to. В идеале нужно нужно продолжить работу по измененным данным
df_bd.drop(['salary_from', 'salary_to'], axis=1, inplace=True)

# Подсчитываем общее количество вакансий по всем группам
df_bd.groupby('experience')['Средняя ЗП'].count().sum()

D:\Temp\ipykernel_15508\262457643.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)
D:\Temp\ipykernel_15508\262457643.py:74: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_bd = pd.read_sql_query(query2, connection)


np.int64(480)

**В результате преобразований у нас получилось выделить 480 вакансий из 536 которые можем использовать в выборке. Мы буквально переврнули проценты, из 84% неподходящих стало 89% подходящих!**

### 2. Следующим фактором по популярности - Стабильность/устойчивость компании.

**Если организация в состоянии свободно распоряжаться своими деньгами, эффективно их использовать, если у нее имеется налаженный механизм постоянного производства и продажи услуг или товаров, то мы можем считать такую организацию финансово устойчивой.\
Самым результативным вариантом является получение отчетности компаний в открытом доступе на сайте Федеральной Налоговой Службы через БФО
Провести анализ каждой компании и отсортировать их по уровню надежности.**

**(Другим вариантом было взять статистику самых популярных компаний России и с помощью регулярных выражений разложить столбец с именами работодателей по рейтингу)**

### 3. Удобное расположение.
**Можно добавить дополнительный признак, который будет содержать словарь: название города - координаты.** 

----

**По этим трем признакам в первую очередь и будет идти поиск подходящей вакансии.**